# CBA Policy Portal Sraper
CBA make downloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

Oover come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in div#wrapper
2. Has code to deal with ASPX/DynamicContent
3. Limited to commbroker domain

In [1]:
from  scraper import WebScraper
from loguru import logger
import os 

class CBA(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True) -> None:

        logger.info(f"Scraping {depth}: {url}")
        
        # May want to add  "quarantined-products"
        if self.skip(url, depth, exclude=["Logon.aspx"]):
            return
             
        _, extension = self.get_path(url)
        assert extension in [".html", ".aspx"], f"Invalid extension '{extension}' for {url}"

        # If aspx find the HTML url it refers to
        if extension == ".aspx":
            html_url = self.aspx_link_to_html_link(url)
            if not html_url:
                return
        else:
            html_url = url

        # read the contents of the html_url to the original url
        # so ASPX paths are used for the HTML
        html_content = self.fetch_content(html_url) 
        html_content = self.remove_script(html_content) 
        if not (starting_page or self.search_only):
            self.save_content(url, html_content)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "div#wrapper")

# Download Comm Broker

In [2]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cba = CBA("https://www.commbroker.com.au", 
              cookie="s_cc=true; at_check=true; SMT_TRACK_PP=nb:customerportfolioview; kndctr_FFF9306152D80A5C0A490D45_AdobeOrg_identity=CiY1MzQ4ODc3MDE3MzAyMTkyMTI1MzA4NTU5MDY0NjE3NTIxMzI2NlITCJei08XDMhABGAEqBEFVUzMwAPABl6LTxcMy; AMCV_FFF9306152D80A5C0A490D45%40AdobeOrg=MCMID|53488770173021921253085590646175213266; s_gnr=1736118160971-New; ADRUM=s=1736119164952&r=https%3A%2F%2Fwww.commbroker.com.au%2Fapp%2Fcpv%2Fportfolio-loans%3F0; s_nr=1738999017033; s_sq=%5B%5BB%5D%5D; bm_mi=8DCD5D6B3AB54FCEE060FE453FC2CFE5~YAAQXgNiaN5p5CCVAQAAQVG0JRq5w7Xj9DY/uwmZJbqelAXUjl/mJvUmsii9xPR0ge0SvVPUUTR7PAFGevx8oWGbJ2ejuDY3kVLLHlMvpUCoPc/pZCzqD7XupBZOjGLnc9jNiq4R59JZZEqWxYKuc45y9Nxk8jV7AfSNRzwm4meTJ6JZ5pskMA97z15y/3RUEBoMj/RAg8fELKwiME8Y5mtEIWtuihxP1XIT7pErghK6jXb4wWh2mLgLzeIOSAIZ0NbxLSxoHEPb4byn+Ev1Ap1Q2wUPiO8uSHF8feyAB4NkRJ5VxdbaZTC7h+yrr/9VUOEeKkxlMuQAf3KXYiCiFoYIXesIO3NP3msNaCw05KGnbvjNT4WuyXDz//0A51KFEc1Y6z5gsENwIWPkolsz/+f+Dcj0lRjDv4zUz7h7BTcnzswjyxk=~1; ak_bmsc=4DD9FDB13B98C06C654880BD7F932E0D~000000000000000000000000000000~YAAQRQNiaEDT+BWVAQAAllO0JRoSQOb2C97GXDm7crL4aO2wFcGz+fs0bvbq0SbQLeANMaPu7YeSuhVnIH8XlBGT4pE4EkLSdRuMy8rjJut5N1VkHH/mBDHqNur6JLe8j5sA4vNWY2c1f27khVBbc4Bk1cky9CNVU1M39RyErh8+Hcij3Owu7b7z5JwpvgKi6gBsmQBBUZu+WrYLXEdiAiag7+dkPwH/fDoRtqveEYSDiihG3Q6QzQMqxJoH9gCci3iZUIiQlFiih8SsfKCT5fMN2DDwRaUA4hhTEGc4SNkDQA2yFEgzuqlP9s08rvdOnkWVB6iKQuSrn3HcTEc61m6qYCPQB6icMShsA65hXLCLgSkvmHQtheIRgZ6sayqIxNnsjtTIR5aTdV6K1pshWuvQP4WzS372pfIHjfAA+oMRthBvrIbB4eRDlzyGGEF92OJAadMZ3ZsXakMJzFf6FGijpvlmw9lPQ8IquMACXo2uZwMeciwshSFPwY/wCjv7RWscIvLO9wKTBUGzNrVfmIiP3SjPo9P1ACULwwYnHy7oGNN9/PczXVl0VHJcmubrfMg=; SameSite=None; ASP.NET_SessionId=rk3rbgcwc0ziut2rjdfp23lk; cbid=SZc7-XM9gU0OMOGbwVozRPJa_jWwcXktxESV8xR42YvfMUvPOeWtBMVlPw_xm3DTPjEPQhWu0oML0nEyOQkPmm--G7YGEtVKIZVnFLePm6ZpWBW0DQoT6zUXYIqrXkM982u54RJnJg0jEnjTYrzBLLV-ex824RNS35Aqo2kVyDmTkD5yRmTpmU0TYje-3WSa120XRgJxOhaMLJ-KFJ69OfyUEqM-MdeiAtX8pkpVyazBdl_CVxAddTD5lwhpAeOYd6OK417AdSJtq843nCd63mNiq0mZK7S68vuaTrB6CuPsVkQcL-_fyN8jEfgPkj0lOJEnKsdBDisCa8NEmfNOO4w90LGfMWeWUBa5zpH2lnWlDHDuHPzGsLA2SsLgaKmk7_PVCSjx-iJueElC4DjVdnqy1fHsZiapbMDXBsy237zLRRFXsujlvjzjsgSGRjMt-9QARBFfEi4e8_cu7ZLGM6N8lgqFEpnS1a0lUw37Z368SDiSN5law3uK_vUCBMlI3QCUDhXw1J_LF47Z5VQOyK9AKaHP8cDr2FLyY3ew7U8NUY4be-gisJlokAiO3vTo-brUMZ8XsMkP8IFsxYsq2kvP0VH8boqSSstxDsvjTAuAckF2OM1A7U8C8Dl8bGSVB13SHfnN_JOvNucLdCumLUz82NEW1AbQobnLMjwjhlblMGeyZeftH6RnxVsa-UOy1YXAhiXIPIkIQDbty5OCTJNTCb45KBHbb14aETpd47xNommduKvXprQrrdP_TJg8AqqXhcFClFR2C2R-qlWcXgBknRYoeOq3sr7HfIgxqt0lGyroCt826c_IPRgZUlavxHJEx03QZ6QgmjcTUD0_R-_n6ZKlpLOGr8EiDJD5w1L-TKIilHosYMfFwWQjzXBVckn8sbP46QrUp_k7Kak_dABVoHWzQGcvTLKOv6nwt_bObk1804af_XGCTPY119oBUGk-rOQ1qr7ntPUvKGFqvNApyflruE0F5jw3i5c797TVrlFg2mzX9os9OZYVG7I2WmQJ2iedd9OABVol_b2KICZwpmIVhrABBjkn7SYfAvXK-czzvT0DOWPBv4yOWh1qRQ83ApDSc_uFzV9b6iu21-sEaMfHn4s3WB8N0J4csS4J2UWsG8v_Q3htn-FEzu_TOZ9b1wVBuqMplLqAqQ9flZEQrZuSDfs1DBJNN6SLrrhAzgnO2uV0k3DcgPoKu906xsF2bsB45NDJoU3ZE-WZO6ntxFq5b71JkBHulk2RWvaKK4MpLDhKHCTSacWVJWCcJVD5muh5GU2LF_-F_mpBaHtJ3MR6erxkezxQatrGrQ4AsRSs5UFgX4cJjvBIGnxDwmtyJfaaQ1fzW_rf6zpVwh1lw0LzQaI11unNo0nY49rUGv971wq3e5FRZLsntX1YfTqQQNq8mfZz-3NhaKYHywmmYHxMg62ZvITyTbs8JhL5v9NMhrYqdAXDSyGPnXUqPvayeudbZTl4Ps6XzpuhIPw1qBpW9qu3vZUDHnFdjTI0CF3M-SmZFTWnv-FhyAaE4VjAd9F36D7IVwCt5kCLvCYUD8XxgqTFObiNwpXuloHvdBFeDiNNaJv1VZYpbrLs51wL5x9dHu8-1nqNimGlNiNa2raWW-KaYnRViZWrYYKNzynFX0ZAk_xnoAihwQ7Am1Loyl1D4jqXSaduOKwUcewB4-Dl_iHrDw9d0OnY_WPyPX4ADSqLbyxabDoRQBloZNfFUyqc26E4nqAqbjpScLH5h1SaPR2aJz6-cHScqhTmcdU4m9E25E27cFLeflVCXPsP5Hy9P-Kzl4tD8rTJQw1FsTjvdhOxCY33VBoxNn0moHWn1APyL4GT6ZwKe4uopzz46RguRBb-tS6Q0P_yN_p6e19HHTqmfkhJm6L7NSTYDqjwWjusp1a07YtkgtIs6WF8bTV5Ho2gw4THrK8JytRa2cvzXZWTx6HlkncS1W0Sg_Vw-ksEfU3JGgyaVm0QNHaL5KbVi2S4nqxfato2g3j2QDuDbH489SdqF-qh27BGQsZKl-cNewDzc9P30Zdgr5x3rG7rtCakqPWA5PghfcFnHlEjdXZ01iJd-ozsXvPWQ62GJ_eqEiqrhdNKBOjOQDKanDyI_awnzcrcqLAgXA0Am5Hsuj8FXhDESCHMGxs4lLc194oD6EJiuL3SlmHbRqT1yGpj2it0HmZqyJsA0q1qwngIotbGOHki1z4nYJcKvINkr_L68Ir7VPZ5kYcoa3pR-uXGUvvZ1lWCLuBFIuRfM1a9_KCup5W68X9OHkOXVXXGOvKwI6E5MObe2JzD1QO2FSltI8roSD4SutTAB-szT5pTT895kHLD47cnXb5nsF3dJ1ewEVKYrLT-gNQGwI_jmCQYlAyyeEpR_XZjbcnkjaTLap1fWkgfh4_dEGwYYSpUdcNdCBFl1kcNqsG9P6cgRDDpt18IZHbqigz1b_xa9OHnhUNEdeyhlIuvLUkLbHhNJ7SN75ygWhRZBnxNUfFLmyPAk8r9mkWWkM009W1vdW5sEcZjtmVNigCec2P6BYwMWnv7z2eyg9n8iAXEMR81SPD2MFvw9aMyV_LLR6mHnNTddlYiZ1hUJiWKdP7qKokyhCv0Ln5c3mWG1nDupK4rZNFzwfQqQOvonFuOoXl16Nz_-L6DSrQdba_r-CoEqHK0EqQXLht-UwLfZrcshoqjsgAUM-w_mmcPQhiF5T1BDab2p-Z_sEfVKhs58DNMEkyN6dnC1PvpPZMP6cFCeAE5dT3bNco9N-4pbn-M5TcaEz7Zqn6cN-om48aGwtajr67yh6HChrtqam4GTQY2UL0wdA6uXebCPWT12fwAuEIDTcKpCMPM4cCRhmS72cMPVBZQYEO9eDc9wJZDMWO-nDwOuFRbuujWignRs6D2zNXhR1P2JDJqPLab6UlmquuLlH1A9vmJrnSJR3gNB20KYLCpMRhJRObdz_IED-FOrkD0Driywl-x7frcwz27pJCqlsgu-QKiieKrY-yXLupHsOsji6sq9pzMZpeswG4Fo2IE-ZLMOaPITT_5Fd0e7u2FJMo-G17CqwJ72JOkaVGWnSOjSbIT3mHIuGS3JHrZIy1rEBh-Uro5Cp78cc2HMp4egzPC13zuxuzRVU9k-mSJOzF3IHqrXBwc-d5A98wfBOY9OLw3i1LA3OuQoeiO_ue9jxW6-8Mp_sM6MlOK0qG5uV2luNvrrE-NQD2KEczSUBy4k6uuWmLzhIoxgQ95II_KDqcaVD3jEcFqHW-eRvtLF4R2-p3n1gEPC6cZOLvq-cjqgqU1gGnCleBZj9nN_6L_sYqB0vZ_PLmGQoqd-WOwCeQtYH6QA1PWYkR2bQkdKgUIY_NlgMzBCvN9D0bIY1PYKLvfRd74xq5G647ld0Ik24A3n-bJWq1VF-H3M7uEU6eP0rJy5dnmzh7pezNLJn_AS2RixHskoKmd2SECIC0qyw7Hqg; ADRUM_BTa=R:42|g:3ed421dc-9ca9-4a3d-a495-8ed175396b1b|n:cba-prod_0e85eb6c-1d2c-4a35-a7e2-92db3c234685; htsm=exp=1740096815~acl=/*~hmac=256fb30d1bd73b8f94376436c7dce83f75752507235c2673e5db15dacdeeb80a; bm_sz=9D97B5E4320CBFD75A4F32C3A98D3008~YAAQXgNiaEV75SCVAQAAScm+JRoZyjZmKPZlLW0VCDjfjSELHVqSUZIHGfh/GbyN5AbC2Pg3phj1x1Xh73A3sYpmC42v3YETO7F5xiuSMci1BBzVQk5FDspZl4I6+OTdgMzFeco6Ia82RV6HEezDlVDE2LhjIKZS353gMOks0ahCFmUbECKZLdg4aWE8EwoN8kumjHRFlshSmepx7+rDFs7wBdiHJmMbMRmGmRQ/FfHf/IPbUgVRCs746oZ3Lz4pc+MhpAeK3oZFXlhwnJBsratcVTJxzes3zljtxghH1Bji42gZITq1OXvzw3Ti40ssDyg76151wiuSw+YVpvEegomFoNTDrWh7iRRYO0L5ieEtutHeu4A2FTEM5/svEHjT7sF7T8g5NeogDiVafAXWzboHvr4xR9jTrvB3Ehuuf4y4UpyzqQws~3618361~3682869; bm_sv=16062896BE8A55DBA74D306E48ED9361~YAAQRQNiaCbw+RWVAQAABc2+JRpgwSZbCH439LV4HKNUKgGNJdBxzhMqg8+CW73XAMoUGsLow7lvunbkzxynzR/qwVFCTx9hlfEzRsfB7s3D9U7QQ39RzV3QQgFimq8/kyFZDlXqeCXt9nMO9WcJieDF1ulN2KzcC+PIKXTwRzb7rF4PoHAqBEG61gIkZUZKzP7gLwBYbHLKMStAug/VOA3GRQnDeAsw8F1alZncForiefuVkWrwPSN6OTPmntnFRCfob1yggQ==~1; _abck=83915FC29BBE3837033B19107C89A814~-1~YAAQRQNiaFzw+RWVAQAAitC+JQ2TQ2o/CwdMgo9ODkqJRgZdNmVI/s4wVUKekWyTJZc/HisWB7qfzMsyajoHTds+6BIqrwZWHW43iwNbGg8ghEOCVZYkGjaEdX+GTCAQM7mA0xXuI4wCMYOgI/EPSulXPRxCn6hfs+C3dKnPXjoJM3znmjKCtuV1slsMpG1eUTRXKQ6qTU+U86YVh4v8J75IoneYdwE1893tZe9WRASXlwZcRaWykE3U+p3FWBmcQK989DGFMFDJI+XWNME7ZSCkp0tDFpmTcNSF4MsEwMhADLpnqB3v7jgnlgAQ5sumDBgoBOBdeRd5/DB+wYxW78/1Fu9LDJLiNkjPT1OLVwKJGcMiXDnhK2SorMUeGDr1p8OCU+L+JOpDdnqEGUgQFqGsddXnmC49pZEwcQXLB/gTfJxduXvNWJSSmLnCY5oL6hCHUgioLteaxAKQ/sY=~-1~||0||~-1", 
              download_path= directory_path, 
              search_only=False)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", depth=16)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", depth=16)

print(len(cba.download))

2025-02-21 07:45:09.283 | INFO     | __main__:scrape:12 - Scraping 16: https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx
2025-02-21 07:45:09.717 | INFO     | __main__:scrape:12 - Scraping 15: https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-02-21 07:45:09.718 | INFO     | scraper:skip:130 - Skipping domain https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-02-21 07:45:09.719 | INFO     | __main__:scrape:12 - Scraping 15: https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-02-21 07:45:10.076 | INFO     | scraper:save_content:56 - Downloading https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx to /home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker//Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-02-21 07:45:10.102 | INFO     | __main__:scrape:12 - Scraping 14: https://www.commbroker.com.au/Net/Doc

241


In [3]:
import glob
import os

def find_html_aspx_files(directory, extentions:list):  # Default to current directory
    """Finds HTML and ASPX files in a directory and its subdirectories."""

    all_files = []

    for root, _, files in os.walk(directory):  # Walk through directory and subdirectories
        for file in files:
            if file.endswith(extentions): # Check file extensions
                all_files.append(os.path.join(root, file)) # Add full path

    return all_files

# find_html_aspx_files("policy/CBA/CommBroker", extentions=(".html", ".htm", ".aspx"))